In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Charger le fichier CSV
file_path = 'data\openfoodfacts.csv'
# Charger le fichier CSV dans un DataFrame pandas
df = pd.read_csv(file_path, sep='\t',low_memory=True)  # Assurez-vous de spécifier le bon séparateur s'il est différent de la virgule


<>:2: SyntaxWarning: invalid escape sequence '\o'
<>:2: SyntaxWarning: invalid escape sequence '\o'
C:\Users\luneb\AppData\Local\Temp\ipykernel_25888\997103595.py:2: SyntaxWarning: invalid escape sequence '\o'
  file_path = 'data\openfoodfacts.csv'
C:\Users\luneb\AppData\Local\Temp\ipykernel_25888\997103595.py:4: DtypeWarning: Columns (0,9,15,21,22,23,24,25,29,30,31,32,33,42,43,44,48,50,51,54,65,70,148) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep='\t',low_memory=True)  # Assurez-vous de spécifier le bon séparateur s'il est différent de la virgule


In [ ]:
# Afficher les premières lignes du DataFrame pour voir les données
print(df.head())

#On vérifie que la base est bien chargée

df.sample(5)
print ("Le dataset compte {} lignes et {} variables".format(df.shape[0], df.shape[1]))

# Afficher les noms des colonnes du DataFrame
print(df.columns)

#on veut connaître les valeurs de la colonne countries_fr
valeurs_uniques = df['countries_fr'].unique()
print(valeurs_uniques)

valeurs_uniques_tag = df['countries_tags'].unique()
print(valeurs_uniques_tag)

